In [1]:
import sys
import feelpp as feelpp
import feelpp.quality as q
import feelpp.toolboxes.core as tb
from feelpp.toolboxes.fluid import *


In [2]:
# Initialize Feel++ Environment : MPI, PETSc ...
sys.argv = ['fluid-remesh']
e = feelpp.Environment(
    sys.argv, opts=tb.toolboxes_options("fluid"),
    config=feelpp.localRepository("3sphere"))



In [4]:
def remesh_toolbox(f, hclose, hfar, parent_mesh):
    # 3 spheres
    required_facets = ["CircleLeft", "CircleCenter", "CircleRight"]
    required_elts = ["CirLeft", "CirCenter", "CirRight"]
    Xh = feelpp.functionSpace(mesh=f.mesh())
    n_required_elts_before = feelpp.nelements(
        feelpp.markedelements(f.mesh(), required_elts))
    n_required_facets_before = feelpp.nfaces(
        feelpp.markedfaces(f.mesh(), required_facets))
    new_mesh, cpt = feelpp.remesh(f.mesh(), "gradedls({},{})".format(
        hclose, hfar), required_elts, required_facets, None)
    f.applyRemesh(new_mesh)

In [5]:
# load 3 sphere config file
feelpp.Environment.setConfigFile(
    'cases/swimmers/3-sphere/2d/three_sphere_2D.cfg')

# create toolbox
f = fluid(dim=2, orderVelocity=2, orderPressure=1)
f.init()

# 3 spheres
hfar = 2
hclose = hfar/10

parent_mesh = f.mesh()
remesh_toolbox(f, hclose, hfar, None )
f.startTimeStep()
while not f.timeStepBase().isFinished():
    min_etaq = q.etaQ(f.mesh()).min()
    if min_etaq < 0.4:
        remesh_toolbox(f, hclose, hfar, None)
    if feelpp.Environment.isMasterRank():
        print("============================================================\n")
        print("time simulation: {}s iteration : {}\n".format(
            f.time(), f.timeStepBase().iteration()))
        print("  -- mesh quality: {}s\n".format(min_etaq))
        print("============================================================\n")
    for i in range(2):
        f.solve()
    f.exportResults()
    f.updateTimeStep()


fluid(2,2,1,1)

time simulation: 0.0s iteration : 1

  -- mesh quality: 0.6043539765181066s



time simulation: 0.1s iteration : 2

  -- mesh quality: 0.6056659041808227s



time simulation: 0.2s iteration : 3

  -- mesh quality: 0.6069729365882068s



time simulation: 0.30000000000000004s iteration : 4

  -- mesh quality: 0.6082741990624337s



time simulation: 0.4s iteration : 5

  -- mesh quality: 0.6095692446817161s



time simulation: 0.5s iteration : 6

  -- mesh quality: 0.5833312977490764s



time simulation: 0.6s iteration : 7

  -- mesh quality: 0.4950599761979701s



time simulation: 0.7s iteration : 8

  -- mesh quality: 0.39861844419923637s



time simulation: 0.7999999999999999s iteration : 9

  -- mesh quality: 0.6501037649070366s



time simulation: 0.8999999999999999s iteration : 10

  -- mesh quality: 0.6524879731699649s



time simulation: 0.9999999999999999s iteration : 11

  -- mesh quality: 0.6150112584654621s



time simulation: 1.0999999999999999s iteration : 12

In [10]:
# the current directory is feelppdb/3sphere/np_1/
import pandas as pd
import os
os.getcwd()

df=pd.read_csv('fluid.measures.csv')
df.columns = df.columns.str.replace(' ', '')
pdf=df.iloc[::40]
df.head()



,time,Quantities_body_CircleCenter.fluid_forces_0,Quantities_body_CircleCenter.fluid_forces_1,Quantities_body_CircleLeft.fluid_forces_0,Quantities_body_CircleLeft.fluid_forces_1,Quantities_body_CircleRight.fluid_forces_0,Quantities_body_CircleRight.fluid_forces_1,Quantities_body_CircleCenter.fluid_torques,Quantities_body_CircleLeft.fluid_torques,Quantities_body_CircleRight.fluid_torques,...,Quantities_body_CircleRight.mass_center_1,Quantities_body_CircleCenter.rigid_rotation_angles,Quantities_body_CircleCenter.moment_of_inertia_body_frame,Quantities_body_CircleLeft.rigid_rotation_angles,Quantities_body_CircleLeft.moment_of_inertia_body_frame,Quantities_body_CircleRight.rigid_rotation_angles,Quantities_body_CircleRight.moment_of_inertia_body_frame,Quantities_body_CircleCenter.moment_of_inertia,Quantities_body_CircleLeft.moment_of_inertia,Quantities_body_CircleRight.moment_of_inertia
0,0.0,-6.409951,-3.213832e-06,12.197707,-2.872846e-06,-6.416953,-3.217343e-06,1.666527e-04,1.666527e-04,1.666527e-04,...,1.156403e-07,0.0,0.152861,0.0,0.153194,0.0,0.153194,0.152861,0.153194,0.153194
1,0.1,2.008641,-3.676211e-07,-4.194051,3.669474e-08,2.010836,-3.680226e-07,2.801335e-05,2.801335e-05,2.801335e-05,...,2.859029e-07,0.0,0.152861,0.0,0.153194,0.0,0.153194,0.152861,0.153194,0.153194
2,0.2,0.004134,4.504610e-07,0.004138,8.452882e-07,0.004138,4.509530e-07,-4.608062e-06,-4.608062e-06,-4.608062e-06,...,4.719923e-07,0.0,0.152861,0.0,0.153194,0.0,0.153194,0.152861,0.153194,0.153194
3,0.3,0.021137,5.121291e-07,0.021160,8.965473e-07,0.021160,5.126885e-07,-4.482419e-06,-4.482419e-06,-4.482419e-06,...,6.632824e-07,0.0,0.152861,0.0,0.153194,0.0,0.153194,0.152861,0.153194,0.153194
4,0.4,0.022742,4.388982e-07,0.022767,8.212655e-07,0.022767,4.393776e-07,-8.102498e-07,-8.102498e-07,-8.102498e-07,...,8.593023e-07,0.0,0.152861,0.0,0.153194,0.0,0.153194,0.152861,0.153194,0.153194


In [12]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['time'], y=df.loc[:, 'Quantities_body_CircleCenter.mass_center_0'],
                    mode='lines',
                    name='x position'))
fig.add_trace(go.Scatter(x=pdf['time'], y=pdf.loc[:, 'Quantities_body_CircleCenter.mass_center_0'],
                         mode='markers',
                         name='x position every period'))
fig.show()
